In [4]:
import missingno as msno
from tqdm.notebook import tqdm
from konlpy import *

In [14]:
og_raw_data = pd.read_csv('./bookcontents4.csv', error_bad_lines=False)

b'Skipping line 11: expected 7 fields, saw 11\nSkipping line 12: expected 7 fields, saw 10\nSkipping line 14: expected 7 fields, saw 20\nSkipping line 15: expected 7 fields, saw 11\nSkipping line 20: expected 7 fields, saw 35\nSkipping line 29: expected 7 fields, saw 31\nSkipping line 45: expected 7 fields, saw 9\nSkipping line 52: expected 7 fields, saw 11\nSkipping line 62: expected 7 fields, saw 25\nSkipping line 76: expected 7 fields, saw 35\nSkipping line 84: expected 7 fields, saw 17\nSkipping line 101: expected 7 fields, saw 19\nSkipping line 103: expected 7 fields, saw 11\nSkipping line 114: expected 7 fields, saw 23\nSkipping line 121: expected 7 fields, saw 15\nSkipping line 122: expected 7 fields, saw 11\nSkipping line 153: expected 7 fields, saw 25\nSkipping line 186: expected 7 fields, saw 9\nSkipping line 217: expected 7 fields, saw 50\nSkipping line 227: expected 7 fields, saw 8\nSkipping line 244: expected 7 fields, saw 19\nSkipping line 258: expected 7 fields, saw 86\n

In [15]:
msno.matrix(og_raw_data)

In [16]:
og_raw_data = og_raw_data.drop_duplicates('title',keep='first')

In [18]:
og_raw_data.groupby('tags').count()['id'].sort_values()[::-1][:10]

tags
라이트노벨      1049
판타지         633
한국 시        620
영미 장편소설     597
한국 장편소설     534
추리/미스터리     508
로맨스         458
한국 단편소설     347
일본 장편소설     301
국내도서        217
Name: id, dtype: int64

In [31]:
df=og_raw_data[(og_raw_data['tags'] == "판타지") | (og_raw_data['tags'] == "로맨스") | (og_raw_data['tags'] == "추리/미스터리")]

In [33]:
df.iloc[0]['description']

'기억의 문이 열리고 모험이 시작된다! 한국 독자들이 사랑하는 베스트셀러 작가 베르나르 베르베르의 신작 장편소설 『기억』(전2권)이 프랑스 문학 전문 번역가 전미연의 번역으로 열린책들에서 출간되었다. 꾸준히 신작을 발표해 온 베르베르는 이번에 [기억]이라는 테마로 시공간을 넘나들며 자신만의 독특한 세계관을 확장해 나간다. 주인공 르네 톨레다노는 고등학교에서 역사를 가르치는 교사이다. 그는 센강 유람선 공연장 [판도라의 상자]에 갔다가 퇴행 최면의 대상자로 선택당한다. 최면에 성공해 무의식의 복도에 늘어선 기억의 문을 열 수 있게 된 르네. 문 너머에서 엿본 기억은, 제1차 세계 대전의 전장에서 목숨을 잃은 그의 전생이었다. 최면이 끝난 후에도 너무나 생생하고 강렬한 기억에 시달리던 그는 몸싸움에 휘말려 의도치 않게 사람을 죽이고 경찰에 자수할지 말지 고민하며 초조한 나날을 보내게 된다.                                                                                    「이제, 계단을 시각화해 머리에 떠올린 다음 계단을 따라 내려가세요. 내려갔어요? 지금 무의식의 문 앞에 도착했어요. 문이 보이시죠?」아무것도 안 보이는데.「르네, 제 말 듣고 있어요? 아직 우리랑 같이 있죠? 대답해요. 문이 보이죠?」지금 눈을 뜨면 사람들이 다 날 쳐다보고 있겠지? 적극적으로 최면에 응하지 않으면 분명히 엘로디가 전통 마술만 좋아해 쇼를 망쳤다고 날 원망할 거야. 에이, 까짓것, 노력을 좀 해보자. 방금 뭐라고 했지? 그래, 계단. 계단을 내려가면 뭐가 보인다고? 맞아, [무의식의 문]이라고 했어.최면사가 다시 묻는다.「어때요, 보여요?」그런 것 같기도 하고. 그래, 보이는 것 같아. 이건가? 그런 것 같네.「네. 보여요.」--- pp.18-19또 다른 실험에서 엘리자베스 로프터스는 피험자들을 디즈니랜드로 데려가 구경시켰다. 그러고 나서 감상을 물어보면서 만화 영화 주인공인 벅스 버니와 보낸 시간이 즐거웠냐는 질

## desctiption


In [6]:
desc_data = og_raw_data[['id','description','title']]

In [7]:
desc_data = desc_data.dropna(axis=0)
desc_data.shape

(4384, 3)

## text

In [8]:
text_data = og_raw_data[['id','text','title']]

In [9]:
text_data = text_data.dropna(axis=0)
text_data.shape

(2529, 3)

## desc + text

In [35]:
og_raw_data['text'] = og_raw_data['text'].fillna(' ')

In [36]:
og_raw_data['desc_text'] = og_raw_data['description'] + og_raw_data['text']

In [37]:
desc_text = og_raw_data.dropna(axis=0)
desc_text= desc_text.drop_duplicates('title',keep='first')

In [38]:
desc_text.shape

(4384, 7)

## 형태소 분석

In [52]:
from konlpy.tag import *
# hannanum = Hannanum()
okt = Okt()

In [17]:
desc_data = [desc_data for desc_data in desc_data['description']]
desc_data = tqdm(desc_data)

desc_tokens = [okt.morphs(row) for row in desc_data]
len(desc_tokens)

4384

In [18]:
text_data = [text_data for text_data in text_data['text']]
text_data = tqdm(text_data)

text_tokens = [okt.morphs(row) for row in text_data]
len(text_tokens)

2529

In [216]:
text_data = [text_data for text_data in text_data['text']]
text_data = tqdm(text_data)

text_tokens = [okt.phrases(row) for row in text_data]
len(text_tokens)

2529

In [19]:
desc_text_data = [desc_text_data for desc_text_data in desc_text['desc_text']]
desc_text_data = tqdm(desc_text_data)

text_contents_tokens = [okt.morphs(row) for row in desc_text_data]
len(text_contents_tokens)

4384

In [191]:
desc_text_data = [desc_text_data for desc_text_data in desc_text['desc_text']]
desc_text_data = tqdm(desc_text_data)

text_contents_tokens = [okt.phrases(row) for row in desc_text_data]
len(text_contents_tokens)

4384

In [20]:
import pickle 
## Save pickle
with open("desc_tokens2.pickle","wb") as fw:
    pickle.dump(desc_tokens, fw)

In [21]:
import pickle 
## Save pickle
with open("text_tokens2.pickle","wb") as fw:
    pickle.dump(text_tokens, fw)

In [22]:
import pickle 
## Save pickle
with open("text_contents_tokens2.pickle","wb") as fw:
    pickle.dump(text_contents_tokens, fw)

In [247]:
with open("text_contents_tokens3.pickle",'wb') as fw:
    pickle.dump(text_contents_tokens, fw)

## pickle 파일 로드

In [10]:
import pickle 
## Load pickle
with open("text_tokens2.pickle","rb") as fr:
    text_tokens = pickle.load(fr)
len(text_tokens)
#['a', 'b', 'c']

2529

In [11]:
import pickle 
## Load pickle
with open("desc_tokens2.pickle","rb") as fr:
    desc_tokens = pickle.load(fr)
len(desc_tokens)
#['a', 'b', 'c']

4384

In [12]:
import pickle 
## Load pickle
with open("text_contents_tokens3.pickle","rb") as fr:
    text_contents_tokens = pickle.load(fr)
len(text_contents_tokens)
#['a', 'b', 'c']

4384

## stop words 설정

In [13]:
with open('ko_stop_words.txt') as text:
    lines = text.readlines()

In [14]:
stop_words1 = ['.','(',')',',',"'",'%','-','X',').','.','---','p','"','']

In [15]:
stop_words2 = [ line.replace('\n','') for line in lines]

In [16]:
stop_words = stop_words1 + stop_words2

## 시리즈 형태로 바꿔서 apply 함수 적용

In [17]:
text_tokens_series = pd.Series(text_tokens)
text_tokens = text_tokens_series.apply(lambda x : [w for w in x if w not in stop_words])  
len(text_tokens)

2529

In [18]:
desc_tokens_series = pd.Series(desc_tokens)
desc_tokens = desc_tokens_series.apply(lambda x : [w for w in x if w not in stop_words])  
len(desc_tokens)

4384

In [19]:
text_contents_tokens_series = pd.Series(text_contents_tokens)
text_contents_tokens = text_contents_tokens_series.apply(lambda x : [w for w in x if w not in stop_words])  
len(text_contents_tokens)

4384

## 띄워쓰기가 있는 문장으로 변환

In [20]:
contents_for_vectorize = []

for content in text_contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    contents_for_vectorize.append(sentence)
    
len(contents_for_vectorize)

4384

## CountVectorizer

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1)

In [22]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4384x110458 sparse matrix of type '<class 'numpy.int64'>'
	with 842121 stored elements in Compressed Sparse Row format>

## Tf-itf

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

In [24]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4384x110458 sparse matrix of type '<class 'numpy.float64'>'
	with 842121 stored elements in Compressed Sparse Row format>

In [25]:
num_samples, num_fetures = X.shape
num_samples, num_fetures

(4384, 110458)

## pairwise_distances 구하기

In [26]:
from sklearn.metrics import pairwise_distances

In [27]:
book_matrix = pairwise_distances(X,X,metric='cosine')

In [28]:
book_matrix.shape

(4384, 4384)

In [29]:
def title_idx(data,title):
    a = pd.DataFrame(np.argsort(book_matrix))[0].apply(lambda x : data.iloc[x])
    return a[a['title'] == title].index

In [30]:
og_raw_data[og_raw_data['title'] == '개미 (2)']

,id,title,img,text,description,url
3928,4138,개미 (2),http://image.yes24.com/momo/TopCate408/MidCate...,지하 40층. 많은 개미들이 움직이고 있다. 8월의 더위가 한창 기승을 부리고 있는...,프랑스의 천재 작가 베르나르 베르베르의 〈개미〉는 인간 세계와 개미 세계의 만남과 ...,https://www.yes24.com/Product/Goods/185464


In [39]:
title_idx(desc_text,'개미 (2)')

Int64Index([3589], dtype='int64')

## 책 추천 함수

In [40]:
def check_book(data,idx):
    return og_raw_data[og_raw_data['id'] == data.iloc[idx][0]]

In [41]:
def book_recommender(title, data=desc_text ,tokens=text_contents_tokens, vectorizer='count', metric='euclidean', num=10):
    contents_for_vectorize = []

    for content in text_contents_tokens:
        sentence = ''
        for word in content:
            sentence = sentence + ' ' + word

        contents_for_vectorize.append(sentence)

    if vectorizer == 'count':
        vectorizer = CountVectorizer(min_df=1)
        X = vectorizer.fit_transform(contents_for_vectorize)
    else:
        vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')
        X = vectorizer.fit_transform(contents_for_vectorize)

    book_matrix = pairwise_distances(X, X, metric=metric)

    book_dist_df = pd.DataFrame(np.argsort(book_matrix))[
        0].apply(lambda x: data.iloc[x])
    idx = book_dist_df[book_dist_df['title'] == title].index[0]
    print('벡터라이즈 방식: ', str(vectorizer)[:num], ', 거리 구하는 방식 : ', metric)
    print("="*50)
    for i in np.argsort(book_matrix[idx])[:num]:
        result_df = check_book(data, i)
        print(result_df[['id', 'title', 'url']])

In [42]:
book_recommender('데미안',tokens=text_contents_tokens, vectorizer='tftf')

벡터라이즈 방식:  TfidfVecto , 거리 구하는 방식 :  euclidean
        id title                                          url
1881  1983   데미안  https://www.yes24.com/Product/Goods/9019369
        id           title                                          url
4207  4446  데미안 (10주년 특별판)  https://www.yes24.com/Product/Goods/8416212
        id  title                                          url
4497  4763  게르트루트  https://www.yes24.com/Product/Goods/8372496
        id title                                           url
4177  4409  세 여인  https://www.yes24.com/Product/Goods/89871415
        id   title                                           url
4112  4342  자유의 감옥  https://www.yes24.com/Product/Goods/33140640
        id title                                           url
3737  3939    카뮈  https://www.yes24.com/Product/Goods/86416258
        id         title                                          url
2856  3019  폰더 씨의 위대한 결정  https://www.yes24.com/Product/Goods/4483639
        id   title                   

In [43]:
book_recommender('데미안',tokens=text_contents_tokens, vectorizer='count')

벡터라이즈 방식:  CountVecto , 거리 구하는 방식 :  euclidean
        id title                                          url
1881  1983   데미안  https://www.yes24.com/Product/Goods/9019369
      id title                                          url
175  184  원더보이  https://www.yes24.com/Product/Goods/6269148
        id      title                                         url
1328  1395  한없이 낮은 숨결  https://www.yes24.com/Product/Goods/125298
        id    title                                         url
4138  4368  고요함의 폭력  https://www.yes24.com/Product/Goods/225496
        id title                                           url
4376  4634    외투  https://www.yes24.com/Product/Goods/43606923
        id         title                                           url
1058  1100  이로써 영원히 계속되리  https://www.yes24.com/Product/Goods/13487105
        id  title                                         url
1275  1338  성황당 외  https://www.yes24.com/Product/Goods/334219
        id title                                         

In [44]:
book_recommender('데미안',tokens=text_contents_tokens, vectorizer='count', metric='cosine')

벡터라이즈 방식:  CountVecto , 거리 구하는 방식 :  cosine
        id title                                          url
1881  1983   데미안  https://www.yes24.com/Product/Goods/9019369
        id title                                           url
4177  4409  세 여인  https://www.yes24.com/Product/Goods/89871415
        id                       title  \
4475  4741  토니오 크뢰거 · 트리스탄 · 베니스에서의 죽음   

                                            url  
4475  https://www.yes24.com/Product/Goods/17392  
        id title                                           url
1793  1890  플로리다  https://www.yes24.com/Product/Goods/90071909
        id       title                                           url
2182  2303  아이 앰 필그림 1  https://www.yes24.com/Product/Goods/65298925
        id    title                                           url
4611  4882  바람의 기록자  https://www.yes24.com/Product/Goods/86355310
        id title                                           url
3737  3939    카뮈  https://www.yes24.com/Product/Goods/86416258

In [45]:
book_recommender('페스트', tokens=text_contents_tokens, vectorizer='tfitf', metric='cosine')

벡터라이즈 방식:  TfidfVecto , 거리 구하는 방식 :  cosine
        id title                                         url
3693  3892   페스트  https://www.yes24.com/Product/Goods/150509
        id    title                                           url
3740  3942  초판본 페스트  https://www.yes24.com/Product/Goods/89376072
        id title                                           url
3881  4089    전락  https://www.yes24.com/Product/Goods/23553217
        id title                                           url
3737  3939    카뮈  https://www.yes24.com/Product/Goods/86416258
        id  title                                           url
3996  4214  페스트 1  https://www.yes24.com/Product/Goods/65562781
        id  title                                           url
3987  4204  페스트 2  https://www.yes24.com/Product/Goods/65562804
        id   title                                           url
3657  3856  고마운 마음  https://www.yes24.com/Product/Goods/89232127
        id                 title                                

In [46]:
book_recommender('페스트',vectorizer='tfitf',metric='cosine')

벡터라이즈 방식:  TfidfVecto , 거리 구하는 방식 :  cosine
        id title                                         url
3693  3892   페스트  https://www.yes24.com/Product/Goods/150509
        id    title                                           url
3740  3942  초판본 페스트  https://www.yes24.com/Product/Goods/89376072
        id title                                           url
3881  4089    전락  https://www.yes24.com/Product/Goods/23553217
        id title                                           url
3737  3939    카뮈  https://www.yes24.com/Product/Goods/86416258
        id  title                                           url
3996  4214  페스트 1  https://www.yes24.com/Product/Goods/65562781
        id  title                                           url
3987  4204  페스트 2  https://www.yes24.com/Product/Goods/65562804
        id   title                                           url
3657  3856  고마운 마음  https://www.yes24.com/Product/Goods/89232127
        id                 title                                

In [47]:
book_recommender('페스트',vectorizer='tfitf')

벡터라이즈 방식:  TfidfVecto , 거리 구하는 방식 :  euclidean
        id title                                         url
3693  3892   페스트  https://www.yes24.com/Product/Goods/150509
        id    title                                           url
3740  3942  초판본 페스트  https://www.yes24.com/Product/Goods/89376072
        id title                                           url
3881  4089    전락  https://www.yes24.com/Product/Goods/23553217
        id title                                           url
3737  3939    카뮈  https://www.yes24.com/Product/Goods/86416258
        id  title                                           url
3996  4214  페스트 1  https://www.yes24.com/Product/Goods/65562781
        id  title                                           url
3987  4204  페스트 2  https://www.yes24.com/Product/Goods/65562804
        id   title                                           url
3657  3856  고마운 마음  https://www.yes24.com/Product/Goods/89232127
        id                 title                             

In [48]:
book_recommender('페스트',vectorizer='count')

벡터라이즈 방식:  CountVecto , 거리 구하는 방식 :  euclidean
        id title                                         url
3693  3892   페스트  https://www.yes24.com/Product/Goods/150509
        id                                  title  \
1894  1998  Anne of Green Gables 빨간머리앤 : 영어원서 시리즈   

                                               url  
1894  https://www.yes24.com/Product/Goods/85109772  
        id       title                                          url
3369  3549  남쪽으로 튀어! 1  https://www.yes24.com/Product/Goods/2121029
        id      title                                          url
3342  3522  성전이란 무엇인가  https://www.yes24.com/Product/Goods/9142580
        id      title                                           url
1213  1265  흙색을 닮아 간다  https://www.yes24.com/Product/Goods/89649625
        id      title                                         url
1328  1395  한없이 낮은 숨결  https://www.yes24.com/Product/Goods/125298
        id  title                                          url
2861  3024  가시나무새

In [49]:
book_recommender('페스트',vectorizer='count',metric='cosine')

벡터라이즈 방식:  CountVecto , 거리 구하는 방식 :  cosine
        id title                                         url
3693  3892   페스트  https://www.yes24.com/Product/Goods/150509
        id title                                          url
1152  1202    청혼  https://www.yes24.com/Product/Goods/9284612
        id           title                                           url
2571  2708  하워드 필립스 러브크래프트  https://www.yes24.com/Product/Goods/12258736
        id title                                           url
1828  1927  빅 피쉬  https://www.yes24.com/Product/Goods/82590781
        id    title                                           url
3740  3942  초판본 페스트  https://www.yes24.com/Product/Goods/89376072
        id title                                           url
1793  1890  플로리다  https://www.yes24.com/Product/Goods/90071909
        id    title                                          url
3513  3704  소란한 보통날  https://www.yes24.com/Product/Goods/4813261
        id     title                              

In [180]:
book_recommender('데미안',vectorizer='count')

벡터라이즈 방식:  Count , 거리 구하는 방식 :  euclidean
        id title                                          url
1881  1983   데미안  https://www.yes24.com/Product/Goods/9019369
        id   title                                         url
1512  1592  달과 6펜스  https://www.yes24.com/Product/Goods/135880
        id    title                                          url
3250  3428  애도하는 사람  https://www.yes24.com/Product/Goods/3679403
      id      title                                          url
286  303  퇴마록 국내편 2  https://www.yes24.com/Product/Goods/5637187
    id      title                                          url
24  28  퇴마록 국내편 1  https://www.yes24.com/Product/Goods/5637185
    id            title                                           url
29  33  소설 보다 : 가을 2019  https://www.yes24.com/Product/Goods/84498048
        id      title                                           url
3396  3584  다정검객무정검 4  https://www.yes24.com/Product/Goods/81689503
    id       title                            

In [181]:
book_recommender('데미안',vectorizer='count',metric='cosine')

벡터라이즈 방식:  Count , 거리 구하는 방식 :  cosine
        id title                                          url
1881  1983   데미안  https://www.yes24.com/Product/Goods/9019369
      id      title                                           url
316  334  내게 무해한 사람  https://www.yes24.com/Product/Goods/61778623
        id      title                                         url
4744  5029  눈먼 자들의 도시  https://www.yes24.com/Product/Goods/318766
        id title                                           url
4625  4896  방랑자들  https://www.yes24.com/Product/Goods/80128261
        id       title                                           url
3679  3878  고슴도치의 우아함   https://www.yes24.com/Product/Goods/23059727
        id title                                           url
3954  4169    왕국  https://www.yes24.com/Product/Goods/59158807
        id            title                                          url
4759  5044  제대로 된 시체답게 행동해!  https://www.yes24.com/Product/Goods/5533457
        id           title           

In [134]:
book_recommender('빨강 머리 앤',vectorizer='tfitf')

벡터라이즈 방식:  Tfidf , 거리 구하는 방식 :  euclidean
        id    title                                           url
1462  1541  빨강 머리 앤  https://www.yes24.com/Product/Goods/73008181
        id        title                                           url
1762  1858  초판본 빨강 머리 앤  https://www.yes24.com/Product/Goods/88389984
        id           title                                           url
2226  2347  빨강 머리 앤 미니북 세트  https://www.yes24.com/Product/Goods/76163800
        id                                       title  \
1736  1830  빨강 머리 앤(Anne of Green Gables) 한글판 + 영문판 세트   

                                               url  
1736  https://www.yes24.com/Product/Goods/79261152  
        id             title                                           url
1422  1497  빨강 머리 앤 미니미니북 세트  https://www.yes24.com/Product/Goods/78573144
        id            title                                           url
2423  2550  빨강 머리 앤 미니미니북 3  https://www.yes24.com/Product/Goods/78573079
        id         

In [135]:
book_recommender('빨강 머리 앤',vectorizer='tfitf',metric='cosine')

벡터라이즈 방식:  Tfidf , 거리 구하는 방식 :  cosine
        id    title                                           url
1462  1541  빨강 머리 앤  https://www.yes24.com/Product/Goods/73008181
        id        title                                           url
1762  1858  초판본 빨강 머리 앤  https://www.yes24.com/Product/Goods/88389984
        id           title                                           url
2226  2347  빨강 머리 앤 미니북 세트  https://www.yes24.com/Product/Goods/76163800
        id                                       title  \
1736  1830  빨강 머리 앤(Anne of Green Gables) 한글판 + 영문판 세트   

                                               url  
1736  https://www.yes24.com/Product/Goods/79261152  
        id             title                                           url
1422  1497  빨강 머리 앤 미니미니북 세트  https://www.yes24.com/Product/Goods/78573144
        id            title                                           url
2423  2550  빨강 머리 앤 미니미니북 3  https://www.yes24.com/Product/Goods/78573079
        id            

## 테스트 문장 만들기

In [69]:
new_post = ['아 짬뽕 먹고 싶다']
new_post_tokens = [ okt.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 아 짬뽕 먹고 싶다']

In [70]:
new_post_for_vectorize = [w for w in new_post_for_vectorize if w not in stop_words]

In [71]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

## 유클리드 거리 

In [72]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [73]:
dist = [dist_raw(each, new_post_vec) for each in X]

In [74]:
import operator
dist_dict = dict(enumerate(dist))
sorted_dict = sorted(dist_dict.items(), key=operator.itemgetter(1))

In [75]:
print('Best 5 post is ', sorted_dict[:5])
print('Test post is --> ', new_post)

Best 5 post is  [(770, 1.3873755410593815), (2289, 1.4142135623730943), (2885, 1.4142135623730943), (3839, 1.4142135623730943), (332, 1.4142135623730945)]
Test post is -->  ['아 짬뽕 먹고 싶다']


In [76]:
result_list = []
for num in sorted_dict[:10]:
    result = og_raw_data[og_raw_data['id'] == desc_data.iloc[num[0]]['id']]
    result_list.append(result[['id','title','text','description','url']])
pd.concat(result_list)

,id,title,text,description,url
796,829,놀러 가자고요,"모가 자라는 한 달여 동안, 막내 윤 씨는 지구를 떠나고 싶었다.모의 안부를 궁금해...",“사는 날 동안 얼굴 붉히면서 살 거 뭐 있나.같이 놀러나 댕기자”거침없이 콸콸 쏟...,https://www.yes24.com/Product/Goods/61304917
2512,2645,가족 문제,“내가 젊었을 때는 부모님이 나를 통제하고 내 인생을 망쳤다. 그 덕택에 너희 엄마...,"『적절한 균형』, 『그토록 먼 여행』 으로 인도의 정치와 종교, 사회가 안고 있는 ...",https://www.yes24.com/Product/Goods/12751634
3167,3342,신의 카르테 2,“이지(理智)에 치우치면 모가 난다. 감정에 말려들면 낙오하게 된다.”“고집을 부리...,"“신의 손을 가진 의사는 없어도, 이 병원에는 기적이 일어납니다.”현직 의사가 그리...",https://www.yes24.com/Product/Goods/60602725
4229,4470,죽음의 론도,사망자는 사고가 나기 몇 분 전에 누군가에게 문자메시지를 보냈다. 당신 말이 맞았소...,“그날의 진실을 밝히기 위해 무려 20년을 기다려왔다!”계획은 아주 상세하고 주도면...,https://www.yes24.com/Product/Goods/67464188
338,357,체공녀 강주룡,주룡은 공을 독차지하고 이름을 떨치고 싶은 마음이 없다. 전빈이 언젠가 했던 말처럼...,"“앞으로 너는 네가 바라는 대로 살았으면 좋겠다.”싸우고 고뇌하고, 사랑하며 자기 ...",https://www.yes24.com/Product/Goods/62236935
387,407,변사체로 죽더라도 선탠하고 싶어,멕시코 정부는 미국과 통하는 모든 국경을 폐쇄했다. 미국에서 수만 마리 좀비들이 끊...,"유례없는 바이러스의 창궐, 위기를 맞은 세계 속에서사람의 힘으로, 사람이 살아가는 ...",https://www.yes24.com/Product/Goods/89649266
605,627,대한민국의 몰락과 부활 1,우리 경제는 그동안 온 국민이 피땀을 흘린 결과 여기까지 발전해왔다고 하겠다. 그러...,정책 실패로 인해 세계적으로 고립된 대한민국일본의 기습적인 군사행동으로 독도를 빼앗...,https://www.yes24.com/Product/Goods/90062166
764,794,시간을 파는 상점,엄마는 명치끝이 아프다며 오랫동안 밥을 먹지 못했다. 소금에 절인 것처럼 슬픔에 절...,50만 독자가 선택한 김선영 작가의 베스트셀러리커버 특별판으로 만나다!국내 청소년문...,https://www.yes24.com/Product/Goods/64163317
806,840,국수 세트,1권대흥고을 정신적 기둥인 김사과金司果댁 맞손자로 바둑에 동뜬 솜씨를 보이는 똑똑한...,"구도求道의 작가, 김성동 혼신의 역작, 『국수國手』 27년 만의 완간한국문학의 살아...",https://www.yes24.com/Product/Goods/62114090
809,843,바다만큼의 눈물로 너를 기다렸다,그가 내 등을 선교관 벽 쪽으로 강하게 밀쳤다. 석재 벽에 코를 찧는 사태를 막기 ...,살다 보면 마음만이 아니라 몸 또한 말을 한다.몸 전체가 삶에 말을 거는 것이다.이...,https://www.yes24.com/Product/Goods/62173298
